In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import pandas as pd
from IPython.display import Image
from sklearn.metrics import confusion_matrix
import os
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg16 import  preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, Flatten
from pathlib import Path
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import ParameterGrid

In [ ]:
# Define the CNN model
def create_model(num_feature_maps_cl2):
    model = models.Sequential()

    # Convolutional Layer 1 (CL1)
    model.add(layers.Conv2D(4, (3, 3), strides=(1, 1), activation='relu', input_shape=(224, 224, 3)))

    # Pooling Layer 1 (PL1) - Mean Pooling
    model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Convolutional Layer 2 (CL2)
    model.add(layers.Conv2D(num_feature_maps_cl2, (3, 3), strides=(1, 1), activation='relu'))

    # Pooling Layer 2 (PL2) - Mean Pooling
    model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Flatten the output
    model.add(layers.Flatten())

    # Fully Connected Layer
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.BatchNormalization())
    # Output Layer
    model.add(layers.Dense(5, activation='softmax'))
    return model

In [ ]:
test_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input
)  # VGG16 preprocessin

In [ ]:
download_dir = Path("drive/MyDrive/task_1")
train_data_dir = download_dir / "train"
valid_data_dir = download_dir / "valid"
test_data_dir = download_dir / "test"

testgen = test_generator.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),
    class_mode=None,
    shuffle=False,
    seed=42,
    batch_size = 1,
)


Found 500 images belonging to 5 classes.


In [ ]:
def test_acc(model,testgen,batch):
# Evaluate the model
  test_loss, test_acc = model.evaluate(testgen, steps=(testgen.samples + batch - 1)//batch)
  #print(f'Test accuracy: {test_acc}')
  return test_acc

In [ ]:
# Ensure TensorFlow is using the GPU
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
    print("TensorFlow is using the GPU")
else:
    print("TensorFlow is not using the GPU")

TensorFlow is using the GPU


In [ ]:
# Assuming create_model is defined elsewhere
def create_model2(num_feature_maps_cl2, lr):
    model = create_model(num_feature_maps_cl2)
    optimizer = Adam(learning_rate=lr)
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
# Define the hyperparameter grid
param_grid = {
    'batch_size': [64, 128],
    'epochs': [10, 20],
    'num_feature_maps_cl2': [10, 20],
    'lr': [1e-3, 1e-4]
}

In [ ]:
# Define a function to perform the grid search
def perform_grid_search(param_grid):
    best_score = np.inf
    best_params = None
    best_model = None

    for params in ParameterGrid(param_grid):
        batch_size = params['batch_size']
        epochs = params['epochs']
        num_feature_maps_cl2 = params['num_feature_maps_cl2']
        lr = params['lr']
        train_generator = ImageDataGenerator(
            validation_split=0,
            preprocessing_function=preprocess_input,
        )  # VGG16 preprocessing

        traingen = train_generator.flow_from_directory(
          train_data_dir,
          target_size=(224, 224),
          class_mode="categorical",
          subset="training",
          shuffle=True,
          seed=42,
          batch_size = batch_size ,
        )

        train_generator = ImageDataGenerator(
            validation_split=0.999,
            preprocessing_function=preprocess_input,
        )  # VGG16 preprocessing


        validgen = train_generator.flow_from_directory(
            valid_data_dir,
            target_size=(224, 224),
            class_mode="categorical",
            subset="validation",
            shuffle=True,
            seed=42,
            batch_size = batch_size,

        )
        print(f"Training with params: {params}")

        model = create_model2(num_feature_maps_cl2, lr)

        # Define the ModelCheckpoint callback
        checkpoint_filepath = 'drive/MyDrive/task_1/best_model_task_2.keras'
        checkpoint_callback = ModelCheckpoint(
            filepath=checkpoint_filepath,
            monitor='val_loss',
            save_best_only=True,
            save_weights_only=False,
            mode='min',
            verbose=0
        )

        # Train the model
        history = model.fit(
            traingen,
            epochs=epochs,
            validation_data=validgen,
            steps_per_epoch=len(traingen),
            validation_steps=len(validgen),
            callbacks=[checkpoint_callback],
            verbose=1,
            batch_size=batch_size
        )

        # Get the best validation loss for this model
        val_loss = min(history.history['val_loss'])

        if val_loss < best_score:
            best_score = val_loss
            best_params = params
            best_model = model  # You might want to load the best saved model instead

    print(f"Best score: {best_score} with params: {best_params}")
    return best_model, best_params


In [ ]:
best_model, best_params = perform_grid_search(param_grid)

Found 2001 images belonging to 5 classes.
Found 495 images belonging to 5 classes.
Training with params: {'batch_size': 64, 'epochs': 10, 'lr': 0.001, 'num_feature_maps_cl2': 10}
Epoch 1/10
32/32 [==============================] - 1212s 39s/step - loss: 1.6590 - accuracy: 0.2899 - val_loss: 1.9528 - val_accuracy: 0.2707
Epoch 2/10
32/32 [==============================] - 16s 503ms/step - loss: 1.3544 - accuracy: 0.4388 - val_loss: 1.4748 - val_accuracy: 0.3980
Epoch 3/10
32/32 [==============================] - 13s 415ms/step - loss: 1.0962 - accuracy: 0.5797 - val_loss: 1.5308 - val_accuracy: 0.3212
Epoch 4/10
32/32 [==============================] - 13s 413ms/step - loss: 0.6653 - accuracy: 0.7796 - val_loss: 1.4658 - val_accuracy: 0.4182
Epoch 5/10
32/32 [==============================] - 13s 416ms/step - loss: 0.2934 - accuracy: 0.9305 - val_loss: 1.4566 - val_accuracy: 0.3657
Epoch 6/10
32/32 [==============================] - 13s 412ms/step - loss: 0.1142 - accuracy: 0.9880 - val

KeyboardInterrupt: 

In [ ]:
from tensorflow.keras.models import load_model

best_model = load_model('drive/MyDrive/task_1/best_model_task_2.keras)

In [ ]:
test_acc(best_model,testgen,1)